In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn.preprocessing as preprocessing
import sklearn.model_selection as model_selection
import matplotlib.pyplot as plt

races_df = pd.read_csv(r"races.csv", delimiter=",", header=0, index_col='race_id')
races_df = races_df[['venue', 'config', 'surface', 'distance', 'going', 'race_class']]

In [10]:
print(races_df)

        venue config  surface  distance         going  race_class
race_id                                                          
0          ST      A        0      1400  GOOD TO FIRM           5
1          ST      A        0      1200  GOOD TO FIRM           5
2          ST      A        0      1400  GOOD TO FIRM           4
3          ST      A        0      1200  GOOD TO FIRM           1
4          ST      A        0      1600  GOOD TO FIRM           4
...       ...    ...      ...       ...           ...         ...
6344       ST      A        0      1400          GOOD           1
6345       ST      A        0      1600          GOOD          11
6346       ST      A        0      2000          GOOD          11
6347       ST      A        0      1200          GOOD           2
6348       ST      A        0      1600          GOOD           2

[6349 rows x 6 columns]
